In [1]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import scipy
from sympy.physics.mechanics import *

In [14]:
o_x,  o_z,  phi,  theta,  psi, v_x, v_y, v_z, w_x, w_y, w_z = dynamicsymbols(r'o_x o_z \phi \theta \psi, v_x, v_y, v_z, w_x, w_y, w_z')
o_xd,  o_zd,  phid,  thetad,  psid, v_xd, v_yd, v_zd, w_xd, w_yd, w_zd = dynamicsymbols(r'o_x o_z \phi \theta \psi, v_x, v_y, v_z, w_x, w_y, w_z', 1)
o_xdd,  o_zdd,  phidd,  thetadd,  psidd, v_xdd, v_ydd, v_zdd = dynamicsymbols(r'o_x o_z \phi \theta \psi, v_x, v_y, v_z', 2)

In [35]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

R_1in0 = Rz * Ry * Rx
v_01in1 = sym.Matrix([v_x, v_y, v_z])
w_01in1 = sym.Matrix([w_x, w_y, w_z])
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([0, 0, 1]),
                              (Rx).T * sym.Matrix([0, 1, 0]),
                                       sym.Matrix([1, 0, 0]))

N = sym.simplify(Ninv.inv())
m, J_x, J_y, J_z, g, m_pen, l_pen = sym.symbols('m, J_x, J_y, J_z, g, m_pen, l_pen')
# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')
f_in1 = R_1in0.T * sym.Matrix([0, 0, -m * g]) + sym.Matrix([0, 0, f_z])


Matrix([
[                   (g*m*sin(\theta(t)) + m*v_y(t)*w_z(t) - m*v_z(t)*w_y(t))/m],
[     (-g*m*sin(\phi(t))*cos(\theta(t)) - m*v_x(t)*w_z(t) + m*v_z(t)*w_x(t))/m],
[(f_z - g*m*cos(\phi(t))*cos(\theta(t)) + m*v_x(t)*w_y(t) - m*v_y(t)*w_x(t))/m]])

In [29]:
o_x_ddot = sym.diff(R_1in0 * v_01in1)[0]
o_y_ddot = sym.diff(R_1in0 * v_01in1)[1]
o_z_ddot = sym.diff(R_1in0 * v_01in1)[2]

In [33]:
psi_dot = (N * w_01in1)[0]
theta_dot = (N * w_01in1)[1]
phi_dot = (N * w_01in1)[2]
theta_dot

w_y(t)*cos(\phi(t)) - w_z(t)*sin(\phi(t))

In [36]:
v_x_dot = ((1 / m) * (f_in1 - w_01in1.cross(m * v_01in1)))[0]
v_y_dot = ((1 / m) * (f_in1 - w_01in1.cross(m * v_01in1)))[1]
v_z_dot = ((1 / m) * (f_in1 - w_01in1.cross(m * v_01in1)))[2]

In [42]:
o_x_ddot = sym.simplify(o_x_ddot.subs([(psid, psi_dot), (phid, phi_dot), (thetad, theta_dot), (v_xd, v_x_dot), (v_yd, v_y_dot), (v_zd, v_z_dot)]))
o_x_ddot

f_z*(sin(\phi(t))*sin(\psi(t)) + sin(\theta(t))*cos(\phi(t))*cos(\psi(t)))/m

In [44]:
o_z_ddot = sym.simplify(o_z_ddot.subs([(psid, psi_dot), (phid, phi_dot), (thetad, theta_dot), (v_xd, v_x_dot), (v_yd, v_y_dot), (v_zd, v_z_dot)]))
o_z_ddot

f_z*cos(\phi(t))*cos(\theta(t))/m - g